In [ ]:
from langchain_community.llms import Ollama
from langchain.embeddings import OllamaEmbeddings

In [ ]:
llm = Ollama(model = "llama3")
embedding = OllamaEmbeddings(model="nomic-embed-text")

# Document Loading

In [ ]:
from langchain.document_loaders import CSVLoader
filePath = "parking_spots.csv"

loader = CSVLoader(file_path=filePath)
data = loader.load()

# Document Extract

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
t_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 0
)

splits = t_splitter.split_documents(data)

# Document VectorStore

In [ ]:
# Method 1
from langchain.vectorstores import Chroma

presist_directory = "docs/chroma/"

vectordb = Chroma.from_documents(
    documents= splits, 
    embedding= embedding,
    persist_directory= presist_directory
)

In [ ]:
# Method 2
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding= embedding
).from_loaders([loader])

# Document Retreival

In [ ]:
question = "get me empty nomal parking near to lift"

In [ ]:
result = vectordb.similarity_search(question)
result

In [ ]:
from IPython.display import display, Markdown

indexResult = index.query(question, llm=llm)
display(Markdown(indexResult))

# Document QnA

In [176]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

chat_history = []

memory = ConversationBufferMemory()
qa = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=index.vectorstore.as_retriever(search_kwargs={"k": 5}), 
    return_source_documents=True,
    return_generated_question=True,
    verbose=True,
)

In [ ]:
question = "Looking for Empty parking"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

In [ ]:
question = "Looking for VIP Empty parking which is near to lift on floor 1"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

In [ ]:
question = "Looking for VIP Empty parking which is near to lift"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

In [ ]:
question = "Looking for Normal Empty parking which is near to lift"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

In [ ]:
question = "Looking for Empty parking which is near to lift"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

In [ ]:
question = "find parking spots floor 2 and 3 near to lift"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

In [179]:
question = "Before answring the question sort data by floor and sort the answer by parking number, How many empty parking available on 1st Floor?"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Floor: 1
Parking Number: 256
Parking Status: Empty
Parking Type: VIP
Metadata: Near to Stairs

Floor: 1
Parking Number: 911
Parking Status: Empty
Parking Type: VIP
Metadata: Near to Door

Floor: 3
Parking Number: 4
Parking Status: Empty
Parking Type: VIP
Metadata: Near to Stairs

Floor: 1
Parking Number: 420
Parking Status: Empty
Parking Type: Normal
Metadata: Right Corner

Floor: 1
Parking Number: 750
Parking Status: Empty
Parking Type: Normal
Metadata: Right Corner

Question: Before answring the question sort data by floor and sort the answer by parking number, How many empty parking available on 1st Floor?
Helpful Answer:

> Finished chain.

> Finished chain.


Before answering the question, I will sort the data by floor. Here is the sorted data:

**Floor 1**

* Parking Number: 256, Parking Status: Empty, Parking Type: VIP, Metadata: Near to Stairs
* Parking Number: 420, Parking Status: Empty, Parking Type: Normal, Metadata: Right Corner
* Parking Number: 750, Parking Status: Empty, Parking Type: Normal, Metadata: Right Corner
* Parking Number: 911, Parking Status: Empty, Parking Type: VIP, Metadata: Near to Door

Now, I will sort the answer by parking number. Here are the results:

There are 4 empty parking available on 1st Floor.